In [7]:
!pip install openpyxl


In [8]:
import openpyxl
# import warnings
from config import CONFIG

In [9]:
# warnings.simplefilter("ignore", category=UserWarning)
debug = True

In [10]:
print("Opening sheet ...")
wb = openpyxl.load_workbook(CONFIG["excelfile"])
sheet1 = wb[CONFIG["sheetname_source"]]
sheet2 = wb[CONFIG["sheetname_dest"]]

Opening sheet ...


/home/ubuntu/.pyenv/versions/3.11.4/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/home/ubuntu/.pyenv/versions/3.11.4/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [11]:
# import os
# from google.colab import files


# excel_path = os.path.join(os.getcwd(), CONFIG["excelfile"])

# if not os.path.isfile(excel_path):
#     print("Excel file not found.")
#     print("Please upload the Excel file.")

#     # Prompt the user to upload the Excel file
#     uploaded_files = files.upload()

#     # Save the uploaded file
#     for filename in uploaded_files.keys():
#         os.rename(filename, excel_path)

#         wb = openpyxl.load_workbook(filename)
#         sheet1 = wb[CONFIG["sheetname_source"]]
#         sheet2 = wb[CONFIG["sheetname_dest"]]

#     print("Excel file uploaded and saved successfully.")



In [12]:
print("Parsing and writing data ...")
start_row = 2
for row_no, row_val in enumerate(sheet1.iter_rows(min_row=start_row, max_row=2, values_only=True)):
    if debug:
        print(f"- looping through record #{row_no+start_row}:")
    newsheet = wb.copy_worksheet(sheet2)
    if debug:
        print(f"\t - set title to: {row_val[CONFIG['sysname_col']] + CONFIG['sysname_col_append']}:")
    newsheet.title = row_val[CONFIG["sysname_col"]].rstrip() + CONFIG["sysname_col_append"]

    for target_cell, source_col_str in CONFIG["field_mapping"].items():
        # Read the value in "All Services" sheet in current row + config column
        source_value = ""
        if isinstance(source_col_str, str):
            source_col_val = openpyxl.utils.column_index_from_string(source_col_str)
            if debug:
                print(f"\t - sheet1.cell(row={row_no+start_row}, column={source_col_val}).value")
            source_value = sheet1.cell(row=row_no + start_row, column=source_col_val).value
            if source_value is None:
                source_value = ""

            if target_cell in CONFIG["pre_fills"] and source_value:
                source_value = CONFIG["pre_fills"][target_cell] + str(source_value)

            if debug:
                print(f"\t - reading AllSystems[row={row_no + start_row}, column={source_col_val}]  =  {source_value}.")

        else:  # list of str
            if debug:
                print(f"\t - detected a source list {source_col_str}:")

            source_value = "'"
            for col in source_col_str:
                source_col_val = openpyxl.utils.column_index_from_string(col)
                cell = sheet1.cell(row=row_no + start_row, column=source_col_val).value
                if cell is not None:
                    source_value += "- " + cell + "\n"

        # Write value in new sheet
        if debug:
            print(f"\t=Setting {newsheet.title} sheet cell [{target_cell}] to value: {source_value}.")
        if debug:
            print()
        newsheet[target_cell] = str(source_value).rstrip()


Parsing and writing data ...
- looping through record #2:
	 - set title to: Access Control  Service:
	 - sheet1.cell(row=2, column=6).value
	 - reading AllSystems[row=2, column=6]  =  HQ Access Control System.
	=Setting Access Control Service sheet cell [E5] to value: HQ Access Control System.

	 - detected a source list ['E', 'F', 'G', 'I']:
	=Setting Access Control Service sheet cell [C8] to value: '- Suprema 
- HQ Access Control System
- Cloud
.

	 - sheet1.cell(row=2, column=11).value
	 - reading AllSystems[row=2, column=11]  =  Bin Hindi .
	=Setting Access Control Service sheet cell [H11] to value: Bin Hindi .

	 - sheet1.cell(row=2, column=12).value
	 - reading AllSystems[row=2, column=12]  =  BHD 1000.
	=Setting Access Control Service sheet cell [H8] to value: BHD 1000.

	 - detected a source list ['S', 'T', 'U', 'V']:
	=Setting Access Control Service sheet cell [J8] to value: '- Avecto Defend Point
- Biometric Access Device to the Server Room
- on-premises components
.

	 - she

In [13]:
print("Cleaning ...")
# Close the workbook
# wb.close()
wb.save(CONFIG["modified_excelfile"])
wb.close()

print("Done.")

Cleaning ...
Done.
